#  Support Vector Machine (SVM) multi

### 1) Import Data

Unduh dataset yang akan digunakan pada tugas kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('day1 IRIS.csv')



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [2]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
data['species'].value_counts()

Iris-setosa        50
Iris-versicolor    50
Iris-virginica     50
Name: species, dtype: int64

## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [4]:
from sklearn.model_selection import train_test_split
data_latih,data_uji = train_test_split(data,test_size=0.2)

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [5]:
print("data uji",data_uji.shape[0])
print("data latih",data_latih.shape[0])

data uji 30
data latih 120


Pisahkan label/kelas dari data uji menjadi sebuah variabel bernama **label_uji**

In [6]:
label_uji = data_uji.pop('species')

## 3) Pembentukan data latih one-vs-rest

Metode one-vs-rest memerlukan tiga jenis data latih yang diperlukan untuk melatih tiga SVM yang berbeda pada dataset Iris. Fungsi **buat_trainingset** digunakan untuk membentuk tiga dataset tersebut.

In [7]:
def buat_trainingset(dataset):
  trainingset = {}
  kolom_kelas = dataset.columns[-1]
  list_kelas = dataset[kolom_kelas].unique()
  print(kolom_kelas)
  print(list_kelas)
  for kelas in list_kelas:
    data_temp = dataset.copy(deep=True)
    data_temp[kolom_kelas]=data_temp[kolom_kelas].map({kelas:1})
    data_temp[kolom_kelas]=data_temp[kolom_kelas].fillna(-1)
    trainingset[kelas]=data_temp
  return trainingset

Gunakan fungsi **buat_trainingset** untuk membentuk data latih dengan nama variabel **trainingset** yang akan digunakan pada proses training. 

In [8]:
trainingset = buat_trainingset(data_latih)  

species
['Iris-versicolor' 'Iris-setosa' 'Iris-virginica']


Tampilkan isi **trainingset** agar Anda dapat memahami struktur dari variabel tersebut.

In [9]:
print(trainingset)

{'Iris-versicolor':      sepal_length  sepal_width  petal_length  petal_width  species
79            5.7          2.6           3.5          1.0      1.0
39            5.1          3.4           1.5          0.2     -1.0
24            4.8          3.4           1.9          0.2     -1.0
90            5.5          2.6           4.4          1.2      1.0
66            5.6          3.0           4.5          1.5      1.0
..            ...          ...           ...          ...      ...
4             5.0          3.6           1.4          0.2     -1.0
105           7.6          3.0           6.6          2.1     -1.0
69            5.6          2.5           3.9          1.1      1.0
89            5.5          2.5           4.0          1.3      1.0
53            5.5          2.3           4.0          1.3      1.0

[120 rows x 5 columns], 'Iris-setosa':      sepal_length  sepal_width  petal_length  petal_width  species
79            5.7          2.6           3.5          1.0     -1.0
39

## 4) Pembentukan SVM Biner

Tujuan dari algoritma SVM adalah meminimalkan nilai *cost function*. Penghitungan nilai minimal dapat dapat dilakukan dengan menghitung nilai gradien dari *cost function* terlebih dahulu. Fungsi di bawah ini berguna untuk menghitung nilai gradien cost function

In [10]:
def hitung_cost_gradient(W,X,Y,regularization):
    jarak = 1 - (Y* np.dot(X,W))
    dw = np.zeros(len(W))
    if max(0,jarak)==0:
        di=W
    else:
        di = W - (regularization * Y*X)
    dw += di
    return dw

Terdapat beberapa cara untuk meminimalkan nilai *cost function*, salah satunya menggunakan Stochastic Gradient Descent (SGD) untuk melakukan minimasi. Minimasi *cost function* merupakan inti dari algoritma SVM. Fungsi di bawah ini merupakan implementasi algoritma SGD 

In [11]:
from sklearn.utils import shuffle

def sgd(data_latih,label_latih,learning_rate = 0.000001,max_epoch=1000,regularization=10000):
  data_latih = data_latih.to_numpy()
  label_latih = label_latih.to_numpy()
  bobot = np.zeros(data_latih.shape[1])
  for epoch in range(1,max_epoch):
    X,Y =shuffle(data_latih,label_latih,random_state=101)
    for index,x in enumerate(X):
      delta=hitung_cost_gradient(bobot,x,Y[index],regularization)
      bobot = bobot - (learning_rate * delta)
  return bobot

## 5) Proses Training

Proses training dilakukan dengan memanggil fungsi **sgd** berulang kali sesuai banyaknya kelas yang ada pada data. Dengan demikian, proses training menghasilkan bobot sebanyak kelas yang ada pada dataset. Buatlah fungsi bernama **training** yang digunakan untuk melakukan proses training one-vs-rest

In [12]:
def training(trainingset):
  list_kelas = trainingset.keys()
  w = {}
  for kelas in list_kelas:
    data_latih = trainingset[kelas]
    label_latih = data_latih.pop(data_latih.columns[-1])
    w[kelas] = sgd(data_latih,label_latih)
  return w

Lakukan proses training dengan memanggil fungsi **training** dan menempatkan hasilnya pada variabel **W**

In [13]:
W = training(trainingset)

Tampilkan isi variabel **W**

In [14]:
print(W)

{'Iris-versicolor': array([ 1.17846984, -2.6543843 ,  1.48164509, -4.53367935]), 'Iris-setosa': array([ 0.16511501,  0.71772235, -1.02092457, -0.54057692]), 'Iris-virginica': array([-2.76064145, -5.27483689,  4.65238406,  6.18513996])}


## 6) Proses *testing* biner
Proses testing dilakukan dengan menghitung nilai [*dot product*](https://en.wikipedia.org/wiki/Dot_product) antara bobot hasil training dengan data uji. Kelas data ditentukan berdasarkan tanda (positif atau negatif) dari hasil dot product tersebut. Fungsi berikut mengimplementasikan proses testing

In [15]:
def testing(W,data_uji):
    prediksi = np.array([])
    for i in range(data_uji.shape[0]):
        y_prediksi = np.sign(np.dot(W,data_uji.to_numpy()[i]))
        prediksi = np.append(prediksi,y_prediksi)
    return prediksi

## TUGAS
Pada tugas kali ini Anda mendefinisikan proses testing pada metode one-vs-rest. Proses testing pada metode one-vs-rest dilakukan dengan memanggil proses testing biner untuk setiap **value** pada dictionary **W**. Kelas pada sebuah data latih adalah **key** pada dictionary **W** yang memiliki nilai prediksi **1**. Lengkapi fungsi **testing_onevsrest** di bawah ini. Output dari fungsi tersebut adalah list nama kelas hasil prediksi.

In [16]:

def testing_onevsrest(W,data_uji):

  list_kelas = W.keys()
  biner = pd.DataFrame(columns=W.keys())
  for kelas in list_kelas:
    biner[kelas]= testing(W[kelas],data_uji) 
  kelas_prediksi= biner.idxmax(1)

  print('Hasil Prediksi:\n',kelas_prediksi)

  return kelas_prediksi

In [17]:
predict = testing_onevsrest(W,data_uji)

Hasil Prediksi:
 0     Iris-versicolor
1      Iris-virginica
2         Iris-setosa
3     Iris-versicolor
4         Iris-setosa
5     Iris-versicolor
6      Iris-virginica
7     Iris-versicolor
8     Iris-versicolor
9      Iris-virginica
10    Iris-versicolor
11        Iris-setosa
12        Iris-setosa
13     Iris-virginica
14    Iris-versicolor
15     Iris-virginica
16    Iris-versicolor
17     Iris-virginica
18        Iris-setosa
19        Iris-setosa
20     Iris-virginica
21    Iris-versicolor
22    Iris-versicolor
23        Iris-setosa
24    Iris-versicolor
25        Iris-setosa
26     Iris-virginica
27        Iris-setosa
28    Iris-versicolor
29     Iris-virginica
dtype: object


Berapa banyak data latih yang berhasil diprediksi dengan benar?

In [18]:
prediks = predict.to_numpy()

In [19]:
print(sum(prediks==label_uji))

26
